1. Load batch data
2. Loop cases
3. Push results back to staging SQLite

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import json
import datetime

In [ ]:
df = pd.read_parquet("data/formation_sample.parquet.gzip")
n = len(df)

In [ ]:
run_name = "olvi"
conn = sqlite3.connect("data/eval.db")
cursor = conn.cursor()
y = cursor.execute(f"SELECT DISTINCT id FROM {run_name}")
records = y.fetchall()
completed = len(records)
completed

In [ ]:
ids = [r[0] for r in records]
print(ids)

In [ ]:
y = cursor.execute(f"SELECT * FROM {run_name} LIMIT 100")
records = y.fetchall()

In [ ]:
def get_output_info(output: str, route: list[str]) -> str:
    """Get the information from the output."""
    response = json.loads(output)
    for r in route:
        response = response[r]

    try:
        return json.loads(response)
    except TypeError:
        return response

Calculate the estimated finish time for the run

In [ ]:
eval_durations = []
for r in records:
    output = r[2]
    eval_durations.append(get_output_info(output, route=["eval_duration"]))
expected_finish_hours = (n - completed) * (np.mean(eval_durations) / 1e9) / 3600 / 6


finish_time = datetime.datetime.now() + datetime.timedelta(hours=expected_finish_hours)
print(f"ETA: {finish_time.strftime('%Y-%m-%d %H:%M:%S')}")

Parsing output to triplet format

In [ ]:
def parse_output(output: str) -> list[dict]:
    """Parse the json output from the record."""
    return get_output_info(output, ["message", "content"])


parse_output(records[0][2])

Drop obsolete testing tables

In [ ]:
# Drop test table
drop_table_name = "test_olvi"
cursor.execute(f"DROP TABLE IF EXISTS {drop_table_name}")